# Surprise 패키지를 이용한 추천시스템 구현

In [10]:
import surprise
print(surprise.__version__)

from surprise import SVD, Reader, Dataset, accuracy 
# SVD 행렬 분해 방식 
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV
from surprise.dataset import DatasetAutoFolds
import pandas as pd
import numpy as np

1.1rc0


### 추천 알고리즘 평가

#### 데이터 준비

In [11]:
# surprise의 Reader 클래스는 헤더가 없는 CSV 형식의 평점 화일을 요구함
#pd.read_csv('ratings.csv').to_csv('ratings_noh.csv', index=False, header=False)

# 평점화일이 user, item, rating, timestamp 순으로 필드가 구성되어 있는다는 것을 알려줘야 함
# 또한 최소 평점과 최대 평점을 지정해야 함
#reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
#data = Dataset.load_from_file('ratings_noh.csv', reader)

In [12]:
ratings = pd.read_csv('ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0)) # Rating 값의 범위에 대하여 알려주어야함. 
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader) 
# 첫번째 Userid 두번째 movieID 세번째 rating으로 고정해야함.  

#### 학습과 평가용으로 데이터 분리

In [13]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0) # Y가 필요X 

#### 추천 알고리즘 학습

In [20]:
model = SVD(n_factors=50, n_epochs=20, biased=True, random_state=0)
# biased값 -> 2가지 존재
## 1. 사용자의 편차
## 2. 아이템의 편차 
model.fit(trainset)

#### 추천 성능 평가

In [21]:
predictions = model.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

#### 교차 검증

In [22]:
model = SVD(random_state=0) 
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8795  0.8801  0.8835  0.8810  0.0018  
MAE (testset)     0.6765  0.6763  0.6805  0.6777  0.0019  
Fit time          5.09    4.23    4.49    4.60    0.36    
Test time         0.24    0.26    0.20    0.23    0.03    


{'test_rmse': array([0.87946112, 0.88014307, 0.8834832 ]),
 'test_mae': array([0.67647762, 0.67630028, 0.68045189]),
 'fit_time': (5.091515302658081, 4.226876735687256, 4.486866235733032),
 'test_time': (0.2353827953338623, 0.2603037357330322, 0.19511723518371582)}

#### 하이퍼 파라미터 튜닝

In [23]:
# 최적화할 파라미터들을 딕셔너리 형태로 지정. 
param_grid = {'n_epochs': [20, 40], 'n_factors': [50, 100] }
# Factor의 갯수가 많으면 과적합. 
# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
grid_search.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(grid_search.best_score['rmse'])
print(grid_search.best_params['rmse'])

0.8772705621621485
{'n_epochs': 20, 'n_factors': 50}


### 추천 아이템 생성 

#### 데이터 준비

In [28]:
# Ratings Matrix 생성
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
rating_movies = pd.merge(ratings, movies, on='movieId')
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title', fill_value=0)

In [10]:
# 학습 데이터 생성
# train_test_split( )으로 분리되지 않는 Dataset에 fit( )을 호출하면 오류 발생
# 전체 데이터를 학습 데이터로 사용하려면 DatasetAutoFolds 클래스를 사용해야 함 
# data_folds = DatasetAutoFolds(ratings_file='ratings_noh.csv', reader=reader)

# 전체 데이터를 학습데이터로 생성함. 
# trainset = data_folds.build_full_trainset()

#### 영화 평점 예측

In [29]:
# model = SVD(n_epochs=20, n_factors=50, random_state=0)
# model.fit(trainset)
# best model found by grid search
model = grid_search.best_estimator['rmse']
# retrain on the whole set A 
trainset = data.build_full_trainset() # 전체데이터를 활용해서 돌리겠다고 지정해주어야함.
model.fit(trainset)

#### Top-N 영화 추천

In [30]:
# 추천 영화의 수 설정
N = 10
uid = 9

In [31]:
#testset = trainset.build_anti_testset()
#predictions = model.test(testset)

ratings_pred = np.zeros((1, ratings_matrix.shape[1])) # 한 User에 대한 Rating을 채워 넣는 것이기 때문에 1행만 필요 
for i in range(ratings_matrix.shape[1]):
    ratings_pred[0,i] = model.predict(str(uid), str(i))[3]
    
ratings_pred = pd.DataFrame(ratings_pred, columns=ratings_matrix.columns)
ratings_pred 
# 9번 User ID를 가진 고객의 Rating 

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
0,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557,...,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557


In [32]:
# id로 지정된 사용자의 모든 영화정보 추출하여 Series로 반환함
# 반환된 user_rating은 영화명(title)을 index로 가지는 Series 객체임 
user_rating = ratings_matrix.loc[uid,:]
    
# user_rating이 0보다 크면 기존에 관람한 영화임. 대상 index를 추출하여 list로 만듬
already_seen = user_rating[user_rating > 0].index.tolist()
   
# list comprehension으로 already_seen에 해당하는 movie는 movies_list에서 제외함
unseen_list = [movie for movie in ratings_matrix.columns.tolist() if movie not in already_seen]

# unseen_list에서 가장 평점이 높은 N개의 영화를 추천함 
recomm_items = ratings_pred.loc[0, unseen_list].sort_values(ascending=False)[:N]
list(recomm_items.index)

['À nous la liberté (Freedom for Us) (1931)',
 'Friends with Benefits (2011)',
 'Friendly Persuasion (1956)',
 'Friend Is a Treasure, A (Chi Trova Un Amico, Trova un Tesoro) (Who Finds a Friend Finds a Treasure) (1981)',
 'Fried Green Tomatoes (1991)',
 'Friday the 13th Part VIII: Jason Takes Manhattan (1989)',
 'Friday the 13th Part VII: The New Blood (1988)',
 'Friday the 13th Part VI: Jason Lives (1986)',
 'Friday the 13th Part V: A New Beginning (1985)',
 'Friday the 13th Part IV: The Final Chapter (1984)']

In [ ]:
# Cross validation 성능이 가장 좋은 놈을 뽑아서 해당 알고리즘을 바탕으로 fix 해두고 추천상품 뽑고 recall, precision 구하면 된다. 

# End